<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_an_e_book_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create an e-book PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

Let's start by installing `borb`

In [1]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 4.5 MB/s 
     |████████████████████████████████| 212 kB 34.2 MB/s 
     |████████████████████████████████| 930 kB 47.7 MB/s 
     |████████████████████████████████| 62 kB 972 kB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for qrcode: filename=qrcode-7.3.1-py3-none-any.whl size=40402 sha256=cd9c22a553abcdfba341d1075ccc00fbae510639e2aa1d9c210f612411a7fe61
  Stored in directory: /root/.cache/pip/wheels/93/d7/39/a4111be2cfb8e679938aa671a37888b6afb1f9e7d748e94492
Successfully built qrcode
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [2]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 5.2 MB/s 


With that out of the way, you can now copy the imports needed to create a basic PDF document.

In [3]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF

from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.layout.text.heading import Heading

from borb.pdf.canvas.color.color import HexColor, X11Color

import typing
import re
from decimal import Decimal

This is the part where it gets fun. You're now going to set up everything to be able to add content to your PDF.

In [4]:
# create empty Document
pdf = Document()

# create empty Page
page = Page()

# add Page to Document
pdf.add_page(page)

# create PageLayout
layout: PageLayout = SingleColumnLayout(page)

Not all fonts can handle all characters. Instead of dealing with this in a more elaborate way, here you'll be using `unidecode` which finds the nearest matching ASCII character for a given non-ASCII character.

In [5]:
def to_ascii(s: str) -> str:
  s_out: str = ""
  for c in s:
    if c == '“' or c == '”' or c == 'â':
      s_out += '"'
    else:
      s_out += unidecode(c)  
  return s_out

Now you can move on to the bulk processing of the text.
You'll start by simply downloading the full text from the gutenberg website.

In [6]:
from unidecode import unidecode

# define which ebook to fetch
url = 'https://www.gutenberg.org/files/863/863-0.txt'

# download text
import requests
txt = requests.get(url).text
print("Downloaded %d bytes of text .." % len(txt))

# split to lines
lines_of_text: typing.List[str] = re.split('\r\n', txt)
lines_of_text = [to_ascii(x) for x in lines_of_text]

# debug
print("This ebook contains %d lines .. " % len(lines_of_text))

Downloaded 361359 bytes of text ..
This ebook contains 8892 lines .. 


You don't really need the first couple of lines of text. They're just the copyright headers Project Gutenberg puts on all these works.

In [7]:
# skip header
header_offset: int = 0
for i in range(0, len(lines_of_text)):
  if lines_of_text[i].startswith("*** START OF THE PROJECT GUTENBERG EBOOK"):
    header_offset = i + 1
    break
while lines_of_text[header_offset].isspace():
  header_offset += 1
lines_of_text = lines_of_text[header_offset:]
print("The first %d lines are the gutenberg header .." % header_offset)

The first 24 lines are the gutenberg header ..


Next, you'll ensure the final copyright/legal header is trimmed as well.

In [8]:
# skip footer
footer_offset: int = len(lines_of_text)
for i in range(0, len(lines_of_text)):
  if "*** END OF THE PROJECT GUTENBERG EBOOK" in lines_of_text[i]:
    footer_offset = i
    break
lines_of_text = lines_of_text[0:footer_offset]
print("The last %d lines are the gutenberg footer .." % (len(lines_of_text) - footer_offset))

The last 0 lines are the gutenberg footer ..


With that out of the way, you can move on to processing the main body of text.

In [ ]:
# main processing loop
i: int = 0
while i < len(lines_of_text):
  
  # process lines
  paragraph_text: str = ""
  while i < len(lines_of_text) and not len(lines_of_text[i]) == 0:
    paragraph_text += lines_of_text[i]
    paragraph_text += " "
    i += 1

  # empty
  if len(paragraph_text) == 0:
    i += 1
    continue

  # space
  if paragraph_text.isspace():
    i += 1
    continue

  # contains the word 'CHAPTER' multiple times (likely to be table of contents)
  if sum([1 for x in paragraph_text.split(' ') if 'CHAPTER' in x]) > 2:
    i += 1
    continue

  # debug
  print("Processing line %d / %d" % (i, len(lines_of_text)))

  # outline
  if paragraph_text.startswith("CHAPTER"):
    print("Adding Header of %d bytes .." % len(paragraph_text))
    try:
      page = Page()
      pdf.add_page(page)
      layout = SingleColumnLayout(page)
      layout.add(Heading(paragraph_text, font_color=HexColor("13505B"), font_size=Decimal(20)))
    except:
      pass
    continue

  # default
  try:
      layout.add(Paragraph(paragraph_text))
  except:
    pass
  
  # default behaviour
  i += 1

Processing line 5 / 8519
Processing line 7 / 8519
Processing line 9 / 8519
Processing line 30 / 8519
Adding Header of 26 bytes ..
Processing line 38 / 8519
Processing line 41 / 8519
Processing line 50 / 8519
Processing line 53 / 8519
Processing line 56 / 8519
Processing line 58 / 8519
Processing line 60 / 8519
Processing line 69 / 8519
Processing line 78 / 8519
Processing line 83 / 8519
Processing line 92 / 8519
Processing line 95 / 8519
Processing line 99 / 8519
Processing line 101 / 8519
Processing line 105 / 8519
Processing line 107 / 8519
Processing line 115 / 8519
Processing line 117 / 8519
Processing line 124 / 8519
Processing line 126 / 8519
Processing line 128 / 8519
Processing line 134 / 8519
Processing line 137 / 8519
Processing line 145 / 8519
Processing line 147 / 8519
Processing line 149 / 8519
Processing line 158 / 8519
Processing line 160 / 8519
Processing line 166 / 8519
Processing line 170 / 8519
Processing line 172 / 8519
Processing line 179 / 8519
Processing line 182

Now, you can store the `Document` as a PDF using the `PDF.dumps` method.

In [ ]:
with open("output.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, pdf)

That's it! You now have a PDF e-book. That's how easy it is to create a PDF using `borb`.